In [1]:

from os import listdir
from os.path import isfile, isdir, join
import shutil
import cv2
import numpy as np
import pycocotools.mask as mask  
from tqdm import tqdm
from pycocotools.coco import COCO
import base64
from detectron2.structures import BoxMode


def decode_base64_rles(coco):
    for ann in coco.dataset['annotations']:
        segm = ann['segmentation']
        if type(segm) != list and type(segm['counts']) != list:
            segm['counts'] = base64.b64decode(segm['counts'])

inpath = "./dataset/train/"  # the train folder download from kaggle
outpath = "./train/"  # the folder putting all nuclei image

images_name = listdir(inpath)
cocoformat = {"licenses":[], "info":[], "images":[], "annotations":[], "categories":[]}


In [2]:

cat = {"id": 1, 
       "name": "nucleus", 
       "supercategory": "nucleus",
      }
cocoformat["categories"].append(cat)

In [3]:
mask_id = 1
for i, im_name in tqdm(enumerate(images_name)):
    t_image = cv2.imread(inpath + im_name + "/images/" + im_name + ".png")
    mask_folder = listdir(inpath + im_name + "/masks/")
    im = {"id": int(i+1), 
          "width": int(t_image.shape[1]), 
          "height": int(t_image.shape[0]), 
          "file_name": im_name + ".png",
         }
    cocoformat["images"].append(im)
    for mask_img in mask_folder:
        t_image = cv2.imread(inpath + im_name + "/masks/" + mask_img, 0)
        if t_image is None:
            print(f'Warning: Mask {mask_img} not found')
            continue
        ret, binary = cv2.threshold(t_image,127,255,cv2.THRESH_BINARY)
        contours, hierarchy = cv2.findContours(binary,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        t_image = t_image.clip(max=1)

        # m = mask.encode(np.asfortranarray(t_image))
        m = mask.encode(np.asarray(t_image,order ="F"))
        area = int(mask.area(m))
        x, y, w, h = mask.toBbox(m)

        # print('Original', m['counts'], '\n')
        # m['counts'] = base64.b64encode(m['counts']).decode('utf-8')
        m['counts'] = m['counts'].decode('utf-8')
        # print('Encoded', m['counts'], '\n')
        
        ann = {"id": int(mask_id), 
               "image_id": int(i+1), 
               "category_id": int(1), 
               "segmentation": m, 
               "area": float(area), 
               "bbox": [int(x), int(y), int(w), int(h)], 
               "bbox_mode":BoxMode.XYWH_ABS, 
               "iscrowd": 0,
              }

        mask_id = mask_id+1
        cocoformat["annotations"].append(ann)
        # t_seg = np.where(t_image[:,:]==255)
        
        '''
        all_seg_in_mask = []
        for s in range(len(contours)):
            seg = []
            for x in range(len(contours[s])):
                seg.append(int(contours[s][x][0][0]))
                seg.append(int(contours[s][x][0][1]))
            all_seg_in_mask.append(seg)
        ann = {"id": int(mask_id), 
               "image_id": int(i) + 1, 
               "category_id": int(1), 
               "segmentation": all_seg_in_mask, 
               "area": float(len(t_seg[0])), 
               "bbox": [int(np.min(t_seg[1])), int(np.min(t_seg[0])), int(np.max(t_seg[1]) - np.min(t_seg[1])), 
                        int(np.max(t_seg[0]) - np.min(t_seg[0]))], 
               "iscrowd": 0,
              }
        mask_id = mask_id+1
        cocoformat["annotations"].append(ann)
        '''
        

0it [00:00, ?it/s]

Original b'^U`23nn09L4L4M1O2N2M2N2O2N1O1O1O11O00O100O10O0100O01O100O1O2O0O100N3N1N3N1N3L4M5JgYkj0' 

Encoded ^U`23nn09L4L4M1O2N2M2N2O2N1O1O1O11O00O100O10O0100O01O100O1O2O0O100N3N1N3N1N3L4M5JgYkj0 

Original b'lmj24Qo0:H2N3M2N2N1O2O000O100O100O10000O01OO20O10O01O100O001O2N1N3N1N4L3MR^dj0' 

Encoded lmj24Qo0:H2N3M2N2N1O2O000O100O100O10000O01OO20O10O01O100O001O2N1N3N1N4L3MR^dj0 

Original b'iRn35Po07H7L3L3L5L3M3N2O0O2O0O20OO100000000001O00001O00001O000O2O2N1N3N1N1O2N2O0O2N2N2M3Ln^Yi0' 

Encoded iRn35Po07H7L3L3L5L3M3N2O0O2O0O20OO100000000001O00001O00001O000O2O2N1N3N1N1O2N2O0O2N2N2M3Ln^Yi0 

Original b'a[m45Qo05K4M3M3M2O1N200O2O0O100O101O001N101O1O00001O2O2L2O1O00O1O101N1O1N2O1O2N1O2M3N2N3L4M4J4MRWWh0' 

Encoded a[m45Qo05K4M3M3M2O1N200O2O0O100O101O001N101O1O00001O2O2L2O1O00O1O101N1O1N2O1O2N1O2M3N2N3L4M4J4MRWWh0 

Original b'ejQ33on07N2BNcQO5Zn0>N2N2ISOPROo0hm0=O001O1O0000000001O00001O000001O0O010000O10001O001O001O0O2O1M3N2N3N1O1N3L:F<CjjPj0' 

Encoded ejQ33on07N2BNcQO5Zn0>N2N2ISOPROo0hm0=O0

1it [00:02,  2.75s/it]

Original b'\\ho?4So02O01O1N\\m[>' 

Encoded \ho?4So02O01O1N\m[> 

Original b'eQTn05So03M4L2N1O1O1O1N1O1O0gh1' 

Encoded eQTn05So03M4L2N1O1O1O1N1O1O0gh1 

Original b'SU[a03Uo0000000000O101N1O13M001O1O0000khe<' 

Encoded SU[a03Uo0000000000O101N1O13M001O1O0000khe< 

Original b'emca05Qo03M2M020001N7J`jd<' 

Encoded emca05Qo03M2M020001N7J`jd< 

Original b'Xkjm05Qo04M3M101N101O0O2N101N3M10000001O000O2O1N2N3KVXO' 

Encoded Xkjm05Qo04M3M101N101O0O2N101N3M10000001O000O2O1N2N3KVXO 

Original b'aoim04So03N1O00000000001O001O1O1N10000001O1N2N2LXR2' 

Encoded aoim04So03N1O00000000001O001O1O1N10000001O1N2N2LXR2 

Original b'RcUe07Qo02M2O1N2N2N1O2N1O2O1002M01000O000000N2N2O1001O1N2M5JYc`8' 

Encoded RcUe07Qo02M2O1N2N2N1O2N1O2O1002M01000O000000N2N2O1001O1N2M5JYc`8 

Original b'hPU>3So05L2N101N3N2M2N2O1O0O2O000O100O1N13N2M2M5_OZRf?' 

Encoded hPU>3So05L2N101N3N2M2N2O1O0O2O000O100O1N13N2M2M5_OZRf? 

Original b'ZXi01Wo02N2N2PQOLhn0;O101O1O1O10O103L1O2N2M3MfeVm0' 

Encoded ZXi01Wo02N2N2PQOLhn0;O101O1O1O10O

2it [00:04,  2.45s/it]

...

Original b'\\Vk;6Qo02N2N100O11O00NFVQO:ln000000000O2N2NfgUb0' 

Encoded \Vk;6Qo02N2N100O11O00NFVQO:ln000000000O2N2NfgUb0 

Original b'ffW;3So05L2N2O001N11O1O2N3M`Snb0' 

Encoded ffW;3So05L2N2O001N11O1O2N3M`Snb0 

Original b'YTVm06Po03M3O1N2O0O101N1O101O00001N1O1O10000010O1O1O2M5KbPb0' 

Encoded YTVm06Po03M3O1N2O0O101N1O101O00001N1O1O10000010O1O1O2M5KbPb0 

Original b'^Ulk07Po02O1N100O1O100O1K40001O01O002O1O1O1O001O2N1N3N1L`ok1' 

Encoded ^Ulk07Po02O1N100O1O100O1K40001O01O002O1O1O1O001O2N1N3N1L`ok1 

Original b'kkfm04on05O1O1O1O1O1O1O001O010O01N01O1M2O3N101O01O00100O2M3M\\[O' 

Encoded kkfm04on05O1O1O1O1O1O1O001O010O01N01O1M2O3N101O01O00100O2M3M\[O 

Original b'R^fg01Vo04L2N1O1O1O100O10000O12N0O1O100O1N2N^bW6' 

Encoded R^fg01Vo04L2N1O1O1O100O10000O12N0O1O100O1N2N^bW6 

Original b'ocab0;mn08H0001O001O2N1O0N5JTVd;' 

Encoded ocab0;mn08H0001O001O2N1O0N5JTVd; 

Original b'Wgf5:mn07I3N2N1N1N21N3M4L4IQS_h0' 

Encoded Wgf5:mn07I3N2N1N1N21N3M4L4IQS_h0 

Original b'`Wob09nn02O1N1O1O1000O10OM6Kn

23it [01:16,  5.50s/it]

Original b'ePl=8on03N0O2O1N1000O00100O2O001N1O3K^mT`0' 

Encoded ePl=8on03N0O2O1N1000O00100O2O001N1O3K^mT`0 

Original b'kiok0e0^n0`0C201O01O3L5G`0AenW2' 

Encoded kiok0e0^n0`0C201O01O3L5G`0AenW2 

Original b'Y`Zl04Ro04L200O01N11O0O101O0101M2Nc\\h1' 

Encoded Y`Zl04Ro04L200O01N11O0O101O0101M2Nc\h1 

Original b'k]em0e0_n06I7N2N33K4K2O2M1N2L5M3Lb]>' 

Encoded k]em0e0_n06I7N2N33K4K2O2M1N2L5M3Lb]> 

Original b'cP_j08Po09H7IO1O1O001N2M2Lkhg3' 

Encoded cP_j08Po09H7IO1O1O001N2M2Lkhg3 

Original b']kcj02Uo02N1O1N2K50O2M3Kgmc3' 

Encoded ]kcj02Uo02N1O1N2K50O2M3Kgmc3 

Original b'gTim03ln09N001001N1N3M301NXe=' 

Encoded gTim03ln09N001001N1N3M301NXe= 

Original b'UZoe09mn0:F3N2O1N100N3O0O10O0O1100N110O10O001O010O1O1N2O001O1O010O001N101O000010O0100O1O100O100O1O100O100O2N10000O1Nkak6' 

Encoded UZoe09mn0:F3N2O1N100N3O0O10O0O1100N110O10O001O010O1O1N2O001O1O010O001N101O000010O0100O1O100O100O1O100O100O2N10000O1Nkak6 

Original b'Vmc>3To02N2O000O1001O001N\\lb?' 

Encoded Vmc>3To02N2O000O1001O001N\lb? 

24it [01:22,  3.44s/it]

Original b'aVV76Qo02N2N2N1001O00001O000000001O2M3MUhif0' 

Encoded aVV76Qo02N2N2N1001O00001O000000001O2M3MUhif0 

Original b'jfo9:ln05K3N001O0010O01O1N101O0N3O000010O2N1O101N1O2N1N2O4K5Kc`ec0' 

Encoded jfo9:ln05K3N001O0010O01O1N101O0N3O000010O2N1O101N1O2N1N2O4K5Kc`ec0 

Original b'^WTi07nn05M3M1O2O1N0O11O1N4M5IhcP5' 

Encoded ^WTi07nn05M3M1O2O1N0O11O1N4M5IhcP5 

Original b'\\fe>3Uo01O0O2O001O0O2O1O0O2N1NDXQO=gn0CYQO=gn03O0:F1N2NgYX?' 

Encoded \fe>3Uo01O0O2O001O0O2O1O0O2N1NDXQO=gn0CYQO=gn03O0:F1N2NgYX? 

Original b'mURf04So03L4M2OO101N_aW8' 

Encoded mURf04So03L4M2OO101N_aW8 



In [ ]:
# import json
# with open('coco_annotations.json', 'w') as outfile:
#     json.dump(cocoformat, outfile)


import json
with open("nucleus_cocoformat_RLE.json", "w") as f:
    json.dump(cocoformat, f)

json_obj = json.dumps(cocoformat, indent=4)
with open("test_RLE.json", "w") as outfile:
     outfile.write(json_obj)

In [ ]:
# copy image to another folder
for f in images_name:
    image = listdir(inpath + f + "/images/")
    shutil.copyfile(inpath + f + "/images/" + image[0], outpath + image[0])